In [55]:
import pandas as pd

In [56]:
df=pd.read_csv(r'C:\Users\sachi\OneDrive\Product Demand Forecasting\DATA\sales_data_500k.csv')

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ProductID      500000 non-null  object
 1   Location       500000 non-null  object
 2   Date           500000 non-null  object
 3   UnitsSold      500000 non-null  int64 
 4   PromoCodeUsed  450000 non-null  object
dtypes: int64(1), object(4)
memory usage: 19.1+ MB


In [58]:
df.sample(10)

,ProductID,Location,Date,UnitsSold,PromoCodeUsed
198657,P0085,L01,2024-09-07,5,No
289279,P0261,L04,2024-11-17,5,No
392089,P0189,L07,2024-12-20,2,No
109873,P0134,L04,2024-09-24,10,Yes
238640,P0439,L07,2024-08-20,8,No
147882,P0192,L01,2024-02-01,6,Yes
70843,P0459,L07,2024-03-07,4,No
385964,P0462,L05,2024-04-03,6,Yes
79421,P0247,L06,2024-01-06,10,No
429284,P0066,L03,2024-05-27,8,Yes


In [59]:
df.nunique()

ProductID        500
Location          10
Date             365
UnitsSold         11
PromoCodeUsed      2
dtype: int64

In [60]:
df.duplicated().sum()

np.int64(4194)

In [61]:
df.drop_duplicates(inplace=True)

In [62]:
df.duplicated().sum()

np.int64(0)

In [63]:
df.isnull().sum()

ProductID            0
Location             0
Date                 0
UnitsSold            0
PromoCodeUsed    49909
dtype: int64

# filling missing data of PromoCodeUsed column

In [64]:
# logic is that, extracting 100 data of PromoCodeUSed for each productId , counting the number of "no" and "yes".. fill the value with most number
sampled_df = df.groupby('ProductID').apply(lambda x: x.sample(min(len(x), 100))).reset_index(drop=True)


C:\Users\sachi\AppData\Local\Temp\ipykernel_1788\3544640990.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('ProductID').apply(lambda x: x.sample(min(len(x), 100))).reset_index(drop=True)


In [65]:
promo_counts = sampled_df.groupby(['ProductID', 'PromoCodeUsed']).size().unstack(fill_value=0)


In [66]:
# Create a dictionary with ProductID as key and majority promoCodeUsed as value
promo_majority = {}

for pid in promo_counts.index:
    if promo_counts.loc[pid, 'Yes'] > promo_counts.loc[pid, 'No']:
        promo_majority[pid] = 'Yes'
    else:
        promo_majority[pid] = 'No'


In [67]:
def fill_promo(row):
    if pd.isnull(row['PromoCodeUsed']):
        return promo_majority.get(row['ProductID'], 'No')  # Default 'No' if ProductID not found
    else:
        return row['PromoCodeUsed']

df['PromoCodeUsed'] = df.apply(fill_promo, axis=1)


In [68]:
print("Missing promoCodeUsed after fill:", df['PromoCodeUsed'].isnull().sum())


Missing promoCodeUsed after fill: 0


In [69]:
df.isnull().sum()

ProductID        0
Location         0
Date             0
UnitsSold        0
PromoCodeUsed    0
dtype: int64

In [70]:
df.duplicated().sum()

np.int64(986)

In [71]:
df.drop_duplicates(inplace=True)

In [72]:
df.sample(10)

,ProductID,Location,Date,UnitsSold,PromoCodeUsed
113153,P0183,L04,2024-12-28,9,Yes
455445,P0490,L03,2024-06-21,6,No
24878,P0356,L04,2024-10-28,2,Yes
498043,P0282,L01,2024-08-20,5,Yes
370992,P0422,L08,2024-10-26,1,No
207076,P0336,L06,2024-11-28,8,Yes
109104,P0251,L07,2024-06-25,3,No
61790,P0365,L04,2024-08-22,9,No
381621,P0408,L07,2024-10-30,0,No
301775,P0214,L04,2024-05-04,1,Yes


In [73]:
df.dtypes

ProductID        object
Location         object
Date             object
UnitsSold         int64
PromoCodeUsed    object
dtype: object

In [74]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [75]:
df.dtypes

ProductID                object
Location                 object
Date             datetime64[ns]
UnitsSold                 int64
PromoCodeUsed            object
dtype: object

In [76]:
negative_units = df[df['UnitsSold'] < 0]

print(f"Total negative UnitsSold rows: {len(negative_units)}")

Total negative UnitsSold rows: 0


# outliers

In [77]:
Q1 = df['UnitsSold'].quantile(0.25)
Q3 = df['UnitsSold'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Outlier rows
outliers = df[(df['UnitsSold'] < lower_bound) | (df['UnitsSold'] > upper_bound)]

print(f"Total outliers in UnitsSold: {len(outliers)}")


Total outliers in UnitsSold: 0


In [78]:
outliers.sort_values(by='UnitsSold', ascending=False).head(10)


,ProductID,Location,Date,UnitsSold,PromoCodeUsed


In [52]:
outliers['ProductID'].value_counts().head(10)


Series([], Name: count, dtype: int64)

In [79]:
df.describe()

,Date,UnitsSold
count,494820,494820.000000
mean,2024-06-30 23:00:17.548199680,5.001128
min,2024-01-01 00:00:00,0.000000
25%,2024-04-01 00:00:00,2.000000
50%,2024-07-01 00:00:00,5.000000
75%,2024-09-30 00:00:00,8.000000
max,2024-12-30 00:00:00,10.000000
std,NaN,3.162425


In [80]:
df.to_csv(r"C:\Users\sachi\OneDrive\Product Demand Forecasting\CLEANED DATA\sales_data_Cleaned.csv",index=False)